# Guide Chatbot

In [28]:
# 라이브러리 빌드
import pandas as pd
from langchain_ollama import ChatOllama
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain_huggingface import HuggingFaceEmbeddings
import gradio as gr
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [29]:
# CSV 파일 로드 및 확인
df = pd.read_csv("guideChatbot.csv")
df = df.dropna(subset=["inputs", "response"])

In [30]:
def preprocess_text(text):
    text = text.strip()  # 양쪽 공백 제거
    # text = text.replace("\n", " ")  # 줄바꿈 제거
    return text

In [31]:
# 데이터 준비
texts = [
    f"질문: {preprocess_text(row['inputs'])}\n답변: {preprocess_text(row['response'])}"
    for _, row in df.iterrows()
]
if not texts:
    raise ValueError("텍스트 데이터가 비어 있습니다. CSV 파일을 확인하세요.")

In [32]:
# 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens"
)

In [33]:
# 벡터 데이터베이스 생성
vectorstore = FAISS.from_texts(texts, embeddings)
# 저장된 문서 수 확인
print(f"벡터 스토어에 저장된 문서 수: {len(vectorstore.docstore._dict)}")
# 저장된 문서 확인
for key, value in vectorstore.docstore._dict.items():
    print(f"Key: {key}, Value: {value}")

벡터 스토어에 저장된 문서 수: 500
Key: 9c2815e8-a9db-4c59-83c6-cc18ed442342, Value: page_content='질문: 어디에서 회원가입을 할 수 있나요?
답변: 회원가입은 우측 상단의 회원가입 버튼을 이용부탁드립니다.'
Key: 39700df5-a072-47af-aff2-49a3408d353c, Value: page_content='질문: 회원가입은 어디로 가면 되나요?
답변: 회원가입은 우측 상단의 회원가입 버튼을 이용부탁드립니다.'
Key: f446919a-0aae-40d0-a600-cd37a846af48, Value: page_content='질문: 가입 절차는 어디서 시작하나요?
답변: 회원가입은 우측 상단의 회원가입 버튼을 이용부탁드립니다.'
Key: 6180ec92-7f0a-453d-9d8b-70cac7489832, Value: page_content='질문: 새 계정을 만들려면 어디로 가야 하나요?
답변: 회원가입은 우측 상단의 회원가입 버튼을 이용부탁드립니다.'
Key: 0c5ddb88-909d-4ffd-bae8-9905140847e0, Value: page_content='질문: 계정을 만들고 싶은데 어디서 하면 될까요?
답변: 회원가입은 우측 상단의 회원가입 버튼을 이용부탁드립니다.'
Key: 0d6380f1-3b28-4526-9f07-aee8b7312cb2, Value: page_content='질문: 회원가입하려면 어디로 들어가야 하나요?
답변: 회원가입은 우측 상단의 회원가입 버튼을 이용부탁드립니다.'
Key: 7622d4aa-6824-4886-b71a-82389c7b4e8e, Value: page_content='질문: 신규 회원 등록은 어디서 하나요?
답변: 회원가입은 우측 상단의 회원가입 버튼을 이용부탁드립니다.'
Key: 26ea79b6-feb8-4bc9-a81a-7f8fd558009d, Value: page_content='질문: 계정을 등록하려면 어느 페이지로 가야 하나요?
답변: 회

In [34]:
# ChatOllama 모델 초기화
llm = ChatOllama(model="gemma2", temperature=0)

In [35]:
# Conversational Retrieval Chain 초기화
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=True
)

In [36]:
# 채팅 함수 정의
def chat(message, history):
    print(f"입력 메시지: {message}")
    print(f"대화 기록: {history}")

    # 대화 기록 준비
    chat_history = [(human, ai) for human, ai in history]

    # 모델 호출
    response = qa_chain({"question": message, "chat_history": chat_history})

    # 검색된 문서 확인
    print(f"검색된 문서: {[doc.page_content for doc in response['source_documents']]}")
    print(f"모델 응답: {response['answer']}")

    sources = set([doc.metadata.get('source', 'Unknown') for doc in response['source_documents']])
    source_info = f"\n\n참고 출처: {', '.join(sources)}" if sources else ""

    return response['answer'] + source_info

In [37]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "로그인은 어디서 해야하나요?",
        "회원가입은 어떻게 해야하나요?",
        "병원 예약은 어떻게 하나요?"
    ],
    title="Asklepios Guide Chatbot",
    description="저희 Asklepios 이용에 궁금한 점이 있으면 언제든지 물어보세요."
)

/Users/jojungon/HospitalReservation_Asklepios/AIService/guideChatbot/.venv/lib/python3.10/site-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [38]:
# 서버 실행
demo.launch(server_port=7862, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7862

To create a public link, set `share=True` in `launch()`.


In [40]:
demo.close()

Closing server running on port: 7862
